In [1]:
import pickle as pkl
import pandas as pd

In [2]:
folder = "../data/mgp_data/"

In [3]:
def read_MGP():
    mgp_nodes = pd.read_csv(folder+"mgpnodeList_with_year_completion_updated_with_mod_dblp_link.csv",
                            sep=",",lineterminator="\n", low_memory=False)
    #mgp_nodes["Year"] =  pd.to_datetime(mgp_nodes["Year"]).dt.year
    mgp_nodes["Year"] = pd.to_datetime(mgp_nodes['Year'], format='%Y',errors = 'coerce').dt.year
    mgp_edges = pd.read_csv(folder+"mgpedgeList_wodup_updated.csv",
                            sep=",",lineterminator="\n", low_memory=False)
    return (mgp_nodes, mgp_edges)

In [4]:
filename = "final_mathscinet_publs_included_remaining_publ_processed.pkl"

In [5]:
mgp_data = read_MGP()

In [6]:
publication = pd.read_pickle(folder+filename)

In [7]:
publication["publication_year"] =  pd.to_datetime(publication["publication_year"])
publication["publication_year"] =  publication["publication_year"].dt.year

In [8]:
mgp_nodes = mgp_data[0]

In [9]:
assert mgp_nodes["Year"].values[0]==1933

In [10]:
mgp_nodes.head(5)

,Id,Name,Year,University,Country,Title,MSC,BIO,MSN,Dblp_Link
0,1,Ernest Willard Anderson,1933.0,Iowa State University,UnitedStates,Statics of Special Types of Homogeneous Elasti...,Mathematics Subject Classification@ 74—Mechani...,NaN,NaN,NaN
1,2,Archie Higdon,1936.0,Iowa State University,UnitedStates,Stresses in Moderately Thick Rectangular Plates,Mathematics Subject Classification@ 74—Mechani...,NaN,NaN,NaN
2,3,Donald Hill Rock,1939.0,Iowa State University,UnitedStates,Finite Strain Analysis in Elastic Theory,NaN,NaN,http://www.ams.org/mathscinet/MRAuthorID/527409,NaN
3,4,Charles Joseph Thorne,1941.0,Iowa State University,UnitedStates,The Approximate Solution of Linear Differentia...,NaN,NaN,http://www.ams.org/mathscinet/MRAuthorID/552938,NaN
4,5,Ralph Harry Tripp,1942.0,Iowa State University,UnitedStates,Statical Equilibrium of Skew and Sector-Shaped...,NaN,NaN,http://www.ams.org/mathscinet/MRAuthorID/555156,NaN


In [11]:
mgp_nodes["mathscinet_id"] = mgp_nodes["MSN"].apply(lambda x : x.rsplit("/")[-1] if type(x)==str else x)
#mathscinet2mgpid = dict(zip(mgp_nodes["mathscinet_id"], mgp_nodes["Id"]))

In [12]:
mgp_nodes["item_type"] = "Thesis"

In [13]:
#mgp_nodes[["Id","mathscinet_id"]].sample(5)

In [14]:
mgp_nodes.columns

Index(['Id', 'Name', 'Year', 'University', 'Country', 'Title', 'MSC', 'BIO',
       'MSN', 'Dblp_Link', 'mathscinet_id', 'item_type'],
      dtype='object')

In [15]:
publication.columns

Index(['author_id', 'title', 'authors', 'msc_code', 'item_type',
       'journal_name', 'publication_year'],
      dtype='object')

In [16]:
columns = ["mathscinet_id","Title","Name","MSC","item_type","University","Year"]

In [17]:
mgp_thesis = mgp_nodes[columns].copy()

In [18]:
rename_columns = {"mathscinet_id": "author_id", "Title":"title", "Name": "authors","MSC":"msc_code","University":"journal_name","Year":"publication_year"}

In [19]:
rename_columns

{'mathscinet_id': 'author_id',
 'Title': 'title',
 'Name': 'authors',
 'MSC': 'msc_code',
 'University': 'journal_name',
 'Year': 'publication_year'}

In [20]:
mgp_thesis = mgp_thesis.rename(columns=rename_columns)

In [21]:
mgp_thesis.columns == publication.columns

array([ True,  True,  True,  True,  True,  True,  True])

In [22]:
mgp_thesis["authors"]= mgp_thesis["authors"].apply(lambda x: [x])

In [23]:
publication.shape

(2703851, 7)

In [24]:
frames = [publication, mgp_thesis]

In [25]:
publ_theis_title = pd.concat(frames)

In [26]:
publ_theis_title.shape

(2972504, 7)

In [27]:
publ_theis_title.to_pickle(folder+"final_mathscinet_publs_included_remaining_publ_processed_with_thesis_title.pkl")

In [ ]:
##### END HERE ######################################################

In [6]:
# publication = pd.read_pickle(folder+"final_mathscinet_publs_included_remaining_publ_processed_with_thesis_title.pkl")

In [83]:
#publ_theis_title[(publ_theis_title["publication_year"]==1970) & (~publ_theis_title["author_id"].isnull())]

In [84]:
#publication[publication["author_id"]=="249464"]

In [194]:
publication.shape

(2703851, 7)

In [195]:
type(publ_theis_title["publication_year"].values[0])

numpy.float64

In [196]:
#publication["publication_year"] =  pd.to_datetime(publication["publication_year"])

In [197]:
#publication["publication_year"].dt.year[0]

In [198]:
type(mgp_nodes["Year"].values[0])

numpy.float64

In [36]:
#mgp_nodes.sample(5)

In [44]:
#mgp_nodes[mgp_nodes["Id"]==23301]

In [45]:
#mgp_nodes[mgp_nodes["mathscinet_id"]=="707853"]

In [199]:
mgpid2_cy = dict(zip(mgp_nodes["Id"],mgp_nodes["Year"]))
mgp_nodes["mathscinet_id"] = mgp_nodes["MSN"].apply(lambda x : x.rsplit("/")[-1] if type(x)==str else x)
mathscinet2mgpid = dict(zip(mgp_nodes["mathscinet_id"], mgp_nodes["Id"]))
#publication["yearwise_titles"]= publication["yearwise_titles"].apply(lambda x : " ".join(x))
publ_theis_title["mgp_id"]= publ_theis_title["author_id"].map(mathscinet2mgpid)

In [200]:
# publication.sample(5)

In [201]:
def constrained_publication_extract(group, publication, years):
    grp_titles = None
    if type(group)==list or type(group)==tuple:
        grp_publ= publication[publication["mgp_id"].isin(group)].copy()
        tmp = []
        advisor_publ = grp_publ[(grp_publ["mgp_id"]==group[0]) & (grp_publ["publication_year"] >= years[0])]
        if advisor_publ.shape[0] > 0: 
            tmp.append(advisor_publ)
        for grp_memeber, year in zip(group[1:],years[1:]):
            advisees_publ = grp_publ[(grp_publ["mgp_id"]==grp_memeber) & (grp_publ["publication_year"] <= year)]
            if advisees_publ.shape[0] > 0:
                tmp.append(advisees_publ)
        if len(tmp) > 0:
            grp_data = pd.concat(tmp)
            grp_data["yearwise_titles"]= grp_data["yearwise_titles"].apply(lambda x : " ".join(x))
            yearwise_grp_publ = grp_data.groupby(["mgp_id","publication_year"])['yearwise_titles'].apply(list).reset_index(name='group_yearwise_titles').copy()
            yearwise_grp_publ.sort_values('publication_year', inplace=True)
            grp_titles = [(" ".join(yearwise), year) for yearwise, year in yearwise_grp_publ[["group_yearwise_titles","publication_year"]].values]
        else:
            grp_titles = []
    return grp_titles

In [202]:
publication1 = publ_theis_title.groupby(["author_id","publication_year"])['title'].apply(list).reset_index(name='yearwise_titles').copy()

In [203]:
publication1["mgp_id"]= publication1["author_id"].map(mathscinet2mgpid)

In [204]:
# publication1["publication_year"] =  pd.to_datetime(publication1["publication_year"])
# publication1["publication_year"] =  publication1["publication_year"].dt.year

In [205]:
publication1.shape

(1390579, 4)

In [206]:
grp = (30480, 77090, 233001, 126605, 135238, 150501)

In [207]:
years = [1994.0, 2003.0, 2018.0, 2006.0, 2009.0, 2006.0]

In [212]:
publication1[publication1["publication_year"]==1970]

,author_id,publication_year,yearwise_titles,mgp_id
189,100035,1970.0,[Pre-measures on lattices of sets.],493.0
540,100075,1970.0,[Interlacing methods and large indecomposables...,32935.0
697,100085,1970.0,"[""Metrically homogeneous sets'': Corrigendum.]",3522.0
921,100115,1970.0,[Harmonic mixed boundary-value problems in com...,32710.0
1002,100125,1970.0,[Конечные депи маркова ],8017.0
...,...,...,...,...
1389665,99880,1970.0,[Imbedding of locally unknotted one-dimensiona...,36445.0
1390038,99920,1970.0,[Distribution theory of spherical distribution...,12800.0
1390293,99965,1970.0,[Axisymmetric buckling of hollow spheres and h...,12037.0
1390420,99980,1970.0,[Extremum principles for irreversible processe...,13411.0


In [209]:
publication1[publication1["author_id"]=="707853"]

,author_id,publication_year,yearwise_titles,mgp_id
1067952,707853,2003.0,[Method of averaging in constructing asymptoti...,77090.0


In [213]:
grp_publ= publication1[publication1["mgp_id"].isin(grp)].copy()

In [214]:
grp_publ

,author_id,publication_year,yearwise_titles,mgp_id
93048,1165748,2016.0,[Graph homology and stability of coupled oscil...,233001.0
93049,1165748,2018.0,"[Stable configurations in social networks., To...",233001.0
93050,1165748,2020.0,[Volume bounds for the phase-locking region in...,233001.0
93051,1165748,2021.0,[Algebraic properties of $\{\text{\cyr\ `E}\}$...,233001.0
93052,1165748,2022.0,[Undecidable arithmetic properties of solution...,233001.0
502960,249464,1988.0,[Logarithmic reduction of the wrapping effect ...,126605.0
502961,249464,2006.0,[Application of uncertainty inequalities to bo...,126605.0
502962,249464,2007.0,[A note on a non-local Kuramoto-Sivashinsky eq...,126605.0
735670,360400,1994.0,[Aspects of Randomness in Nonlinear Wave Propa...,30480.0
1062385,704409,2002.0,[Formation of quasistationary vortex and trans...,150501.0


In [215]:
constrained_publication_extract(grp, publication1, years)

[('Logarithmic reduction of the wrapping effect with application to ordinary differential equations.',
  1988.0),
 ('Aspects of Randomness in Nonlinear Wave Propagation', 1994.0),
 ('Formation of quasistationary vortex and transient hole patterns through vortex merger.',
  2002.0),
 ('Method of averaging in constructing asymptotics of solutions of systems of linear difference equations. Asymptotic behavior of solutions of some linear difference equations with oscillatory decreasing coefficients. Method of Averaging in Constructing Asymptotics of Solutions of Systems of Linear Difference Equations',
  2003.0),
 ('Application of uncertainty inequalities to bound the radius of the attractor for the Kuramoto-Sivashinsky equation. Uncertainty estimates and $L_2$ bounds for the Kuramoto-Sivashinsky equation. Improved L_2 Bounds for the Kuramoto-Sivashinsky Equation',
  2006.0),
 ('Particle Spreading in a Majda model and Eigenvalue Estimation', 2006.0),
 ('On the Stability of Periodic Solutio